In [ ]:
!pip install plotly
!pip install pymysql

     |████████████████████████████████| 43 kB 2.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import pymysql

from datetime import datetime, timedelta
import plotly.express as px

In [ ]:
def get_db():
    db = pymysql.connect(host='35.76.154.105', port=8999, user='root', passwd='multi1203',
                         db='AI_result', charset='utf8')
    cursor = db.cursor(pymysql.cursors.DictCursor)
    sql = 'SELECT * FROM coin'
    cursor.execute(sql)
    result = cursor.fetchall()
    test = pd.DataFrame(result)
    return test

In [ ]:
def pre_data():
    test = get_db()
    pivot = pd.pivot_table(test, index = ['date', 'coin'],
                           columns = 'lang', values = ['num', 's'],
                           fill_value = 0)
    pivot['sum'] = pivot[('num', 'eng')] + pivot[('num', 'kor')]
    pivot['sek'] = pivot[('num', 'eng')] / pivot['sum'] * pivot[('s', 'eng')] +\
                   pivot[('num', 'kor')] / pivot['sum'] * pivot[('s', 'kor')]
    pivot.columns = pivot.columns.droplevel('lang')
    pivot.reset_index(drop=False, inplace=True)
    pivot.drop(['num', 'num', 's', 's'], axis = 1, inplace = True)

    pivot['date'] = pd.to_datetime(pivot['date'])
    pivot['week'] = pivot['date'].dt.weekday

    indexNames = pivot[(pivot['week'] == 5) | (pivot['week'] == 6)].index
    pivot.drop(indexNames, inplace=True)
    
    pivot['sek'] = - pivot['sek']
    pivot['sek'] = pivot['sek'] + 1
    pivot['sek'] = round(pivot['sek'], 3)

    pivot['sum_1'] = pivot.groupby('coin')['sum'].diff()
    pivot['sum_3'] = pivot.groupby('coin')['sum'].diff(3)
    pivot['sum_5'] = pivot.groupby('coin')['sum'].diff(5)
    pivot['s_1'] = pivot.groupby('coin')['sek'].diff()
    pivot['s_3'] = pivot.groupby('coin')['sek'].diff(3)
    pivot['s_5'] = pivot.groupby('coin')['sek'].diff(5)

    pivot.rename(columns = {'sum':'게시물 수', 'sek':'감정분석'}, inplace=True)

    week = datetime.now() - timedelta(days=7)
    week = week.strftime('%Y-%m-%d')

    data = pivot.loc[pivot.date == week]
    data.reset_index(drop=True, inplace=True)
    return data

In [ ]:
def get_plot_1d():
    data = pre_data()
    hovertemplate = []
    for num in range(len(data)):
        num_1 = data.iloc[num, 5]
        s_1 = round(data.iloc[num, 8], 3)
        num_ = data.iloc[num, 2]
        s = round(data.iloc[num, 3], 3)
        hovertemplate.append(f"게시물 수 비교 : {num_1}<br>감정분석 비교 : {s_1}<br>하루동안의 게시물 수 : {num_}<br>하루동안의 감정분석 : {s}")
    fig = px.scatter(data, x='sum_1', y='s_1',
                 color='감정분석', size=data['게시물 수'],
                 range_color=(-1,1),
                 size_max=50, hover_data = None,
                 template='seaborn',
                 labels={'sum_1':'감소                                    <게시물 수>                                    증가',
                         's_1':'부정                           <감정분석>                           긍정'})
    fig.update_layout(
        xaxis=dict(
            showline=True,
            zeroline=True,
            showgrid=False,
            showticklabels=True,          
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=True,
            showline=False,
            showticklabels=True,
        ),
        showlegend=False
    )
    fig.update_coloraxes(colorscale=px.colors.diverging.Portland,
                         colorbar_nticks=3,
                         colorbar_tickcolor='white',
                         colorbar_title='')
    for i in range(data.shape[0]):
        fig.add_annotation(x=data.sum_1[i], y=data.s_1[i], text=data.coin[i],
                 showarrow=False)
    fig.update_traces(hovertemplate=hovertemplate)
    return fig

In [ ]:
get_plot_1d()

In [ ]:
get_plot_1d().write_html('1202_1d.html')

In [ ]:
def get_plot_3d():
    data = pre_data()
    hovertemplate = []
    for num in range(len(data)):
        num_3 = data.iloc[num, 6]
        s_3 = round(data.iloc[num, 9], 3)
        num_ = data.iloc[num, 2]
        s = round(data.iloc[num, 3], 3)
        hovertemplate.append(f"게시물 수 비교 : {num_3}<br>감정분석 비교 : {s_3}<br>하루동안의 게시물 수 : {num_}<br>하루동안의 감정분석 : {s}")
    fig = px.scatter(data, x='sum_3', y='s_3',
                 color='감정분석', size=data['게시물 수'], range_color=(-1,1),
                 size_max=50, hover_data = None,
                 template='seaborn',
                 labels={'sum_3':'감소                                    <게시물 수>                                    증가',
                         's_3':'부정                           <감정분석>                           긍정'})
    fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,          
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=True,
            showline=False,
            showticklabels=True,
        ),
        showlegend=False
    )
    fig.update_coloraxes(colorscale=px.colors.diverging.Portland,
                        #  colorbar_showticklabels=False,
                         colorbar_nticks=3,
                         colorbar_tickcolor='white',
                         colorbar_title='')
    for i in range(data.shape[0]):
        fig.add_annotation(x=data.sum_3[i], y=data.s_3[i], text=data.coin[i],
                 showarrow=False)
    fig.update_traces(hovertemplate=hovertemplate)
    return fig

In [ ]:
get_plot_3d()

In [ ]:
get_plot_3d().write_html('1202_3d.html')

In [ ]:
def get_plot_5d():
    data = pre_data()
    hovertemplate = []
    for num in range(len(data)):
        num_5 = data.iloc[num, 7]
        s_5 = round(data.iloc[num, 10], 3)
        num_ = data.iloc[num, 2]
        s = round(data.iloc[num, 3], 3)
        hovertemplate.append(f"게시물 수 비교 : {num_5}<br>감정분석 비교 : {s_5}<br>하루동안의 게시물 수 : {num_}<br>하루동안의 감정분석 : {s}")
    fig = px.scatter(data, x='sum_5', y='s_5',
                 color='감정분석', size=data['게시물 수'], range_color=(-1,1),
                 size_max=50, hover_data = None,
                 template='seaborn',
                 labels={'sum_5':'감소                                    <게시물 수>                                    증가',
                         's_5':'부정                           <감정분석>                           긍정'})
    fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,          
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=True,
            showline=False,
            showticklabels=True,
        ),
        showlegend=False
    )
    fig.update_coloraxes(colorscale=px.colors.diverging.Portland,
                        #  colorbar_showticklabels=False,
                         colorbar_nticks=3,
                         colorbar_tickcolor='white',
                         colorbar_title='')
    for i in range(data.shape[0]):
        fig.add_annotation(x=data.sum_5[i], y=data.s_5[i], text=data.coin[i],
                 showarrow=False)
    fig.update_traces(hovertemplate=hovertemplate)
    return fig

In [ ]:
get_plot_5d()

In [ ]:
get_plot_5d().write_html('1202_5d.html')